In [1]:
#imports all packages and sets natrual log to ln
import numpy as np
import pandas as pd
from math import e
from scipy.optimize import minimize
from scipy.special import factorial as factorial
from math import gamma
from scipy.stats import poisson
import warnings
warnings.filterwarnings("ignore")
ln = np.log

# Part 1

In [2]:
#reads and displays candy dataset
df = pd.read_csv('C:/Users/anushatc/Downloads/candy(1).csv')
df

,Packs,People
0,0,102
1,1,54
2,2,49
3,3,62
4,4,44
5,5,25
6,6,26
7,7,15
8,8,15
9,9,10


In [3]:
#Question 1.a - Poisson Log Liklihood

def PLL(lmbda,a,b):
    poisson_pd = 0
    for i in range(len(a)):
        poisson_pd += a[i]*ln(poisson.pmf(b[i], lmbda))
    return (-1)*poisson_pd
#Defines Poisson Log Likelihood at a given value of lambda, given data a and b


a = np.array(df[['People']])
b = np.array(df[['Packs']])
lmbda = 1
#All of the above is prep. Setting lambda to an arbitrary value and assigning data to A and B

solnPLL = minimize(
    PLL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
#This calls a minimzation function given A and B, and tells solver to pick a value between .000001 and 1 for lambda

lmbdaPLL = solnPLL.x[0]
MaxPLL= PLL(lmbdaPLL, a, b)
PLLaic = -2*(MaxPLL) + (2)
PLLbic = -2*(MaxPLL) + ln(21)
#We take the optimal lambda out of the minimzation function and feed it through our function, then display results

print("Lambda Value:", lmbdaPLL)
print("Maximum Log Likelihood", MaxPLL)
print("AIC: ", PLLaic)
print("BIC: ", PLLbic)

Lambda Value: 3.9912275835899855
Maximum Log Likelihood [1544.99639045]
AIC:  [-3087.9927809]
BIC:  [-3086.94825846]


In [4]:
#Question 1-b: NBD Model
def NBD(params,b):
    alpha,n = params
    if b==0:
        return (alpha/(alpha+1))**n
    else:
        return (((n+b-1)*1)/(b*(alpha+1))) * NBD(params,b-1)
#Defines function "NBD" that performs NBD calculations for a given set of paramaters and result

def NBDLL(params,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*ln(NBD(params,b[i]))
    return (-1)*nbd
#defines function "NBD Log Likilhood" that runs function NBD on values/paramaters and returns the log-likihood


alpha = 1
n = 1
nbd = 1
#housekeepingfor minimization

solNBD = minimize(
    NBDLL,
    args = (a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
#runs minimization function for two values with given dataset (defined above)

alphaNBD = solNBD.x[0]
nNBD = solNBD.x[1]
params = (alphaNBD,nNBD)
nbd = NBDLL(params, a, b)
NBDaic = -2*(nbd) + (2*2)
NBDbic = -2*(nbd) + 2*ln(21)
#pulls out optimal valyes


print("Alpha: ", alphaNBD)
print("Shape (n): ", nNBD)
print("Maximum Log Likelihood: ", nbd)
print("AIC: ", NBDaic)
print("BIC: ", NBDbic)

Alpha:  0.2499604580273351
Shape (n):  0.9976363196087878
Maximum Log Likelihood:  [1140.02374625]
AIC:  [-2276.04749249]
BIC:  [-2273.95844762]


In [5]:
#Question 1-c: 0-inflated NBD
def piNBD(alpha,n,k,t):
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t))) * piNBD(alpha,n,k-1,t)
#Runs 0-inflated NBD. Formula changes dependtant on if k (packs) is 0

def piNBDLL(params,a,b):
    pi,alpha,n = params
    k0 = pi + ((1-pi)*piNBD(alpha,n,b[0],1))
    ll = a[0]*ln(k0)
    for i in range(1,len(a)):
        ll += a[i]*ln((1-pi)*piNBD(alpha,n,b[i],1))
    return (-1)*ll
#Returns negative log likihood. creates seperate LL for when b = 0, then itttaerates over non-0

solpiNBD = minimize(
    piNBDLL,
    args = (a,b),
    x0 = np.array((1,1,1)),
    bounds=[(0.000001,0.999999),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
#Minimizes function

piNBDpi = solpiNBD.x[0]
alphaNBDpi = solpiNBD.x[1]
nNBDpi = solpiNBD.x[2]
params = piNBDpi, alphaNBDpi, nNBDpi
nbdPi = -1*piNBDLL(params, a, b)
NBDpiAIC = -2*(nbdPi) + (2*3)
NBDpiBIC = -2*(nbdPi) + 3*ln(21)
#Pulls out optimal values

pinames = ['Pi:', 'Alpha:', 'Shape (n):', 'Log Likihood:', 'AIC:', 'BIC']
pivalues = [piNBDpi, alphaNBDpi, nNBDpi, nbdPi, NBDpiAIC, NBDpiBIC]
print(np.c_[pinames, pivalues])
#concacts and prints them


[['Pi:' 0.11310892709001459]
 ['Alpha:' 0.334188078670199]
 ['Shape (n):' 1.5039147037674592]
 ['Log Likihood:' array([-1136.16564087])]
 ['AIC:' array([2278.33128173])]
 ['BIC' array([2281.46484905])]]


In [6]:
#1-d.1: Two - segment finite mixture model
def segments2(params,a,b):
    pi,lam1,lam2 = params
    ll = 0
    for i in range(len(b)):
        ll += a[i]*ln(pi*(poisson.pmf(b[i],lam1)) + (1-pi)*(poisson.pmf(b[i],lam2)))
    return (-1)*ll
#creates function that returns log likelihood given two groups with independant lambdas and a probability of belonging to each group


segments2sol = minimize(
    segments2,
    args = (a,b),
    x0 = np.array((1,1,1)),
    bounds=[(0.000001,0.999999),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

seg2pi = segments2sol.x[0]
seg2lam1 = segments2sol.x[1]
seg2lam2 = segments2sol.x[2]
seg2pi2 = 1-seg2pi
params = seg2pi, seg2lam1, seg2lam2
seg2LL = -1*segments2(params, a, b)
seg2AIC = -2*(seg2LL) + (2*3)
seg2BIC = -2*(seg2LL) + 3*ln(21)
#pull out values, calculate AIC/BIC

pinames = ['P(group1) (pi): ', 'lambda group1: ', 'P(group2) (1-pi): ', 'lambda group2: ', 'Max Log Likelihoood', 'AIC', 'BIC']
pivalues = [seg2pi, seg2lam1, seg2pi2, seg2lam2, seg2LL, seg2AIC, seg2BIC]
print(np.c_[pinames, pivalues])
#concacts and prints them

[['P(group1) (pi): ' 0.29912119939654425]
 ['lambda group1: ' 9.120680856524297]
 ['P(group2) (1-pi): ' 0.7008788006034558]
 ['lambda group2: ' 1.8021548172941908]
 ['Max Log Likelihoood' array([-1188.83282722])]
 ['AIC' array([2383.66565444])]
 ['BIC' array([2386.79922175])]]


In [7]:
def segments3(params,a,b):
    t1,t2,lam1,lam2,lam3 = params
    pi1 = (e**t1)/((e**t1)+(e**t2)+1) # calculating pie values(pie1,pie2,pie3)
    pi2 = (e**t2)/((e**t1)+(e**t2)+1)
    pi3 = (1)/((e**t1)+(e**t2)+1)
    ll = 0
    for i in range(len(b)):
        ll += a[i]*ln(pi1*(poisson.pmf(b[i],lam1)) + pi2*(poisson.pmf(b[i],lam2)) + pi3*(poisson.pmf(b[i],lam3))) # as per the formula given in the slides 
    return (-1)*ll  
#caluclates pi values using theta calcualtions. This ensures pi1+pi2+pi3 = 1.
#then, feeds all b[i], a[i]'s through, with poission and percentages for each group
#returns negative LL


segments3 = minimize(
    segments3,
    args = (a,b),
    x0 = np.array((1,2,1,1,1)),
    bounds=[(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
#minimizes function


theta13 = segments3.x[0]
theta23 = segments3.x[1]
lambda13 = segments3.x[2]
lambda23 = segments3.x[3]
lambda33 = segments3.x[4]
solution3 = -1*segments3.fun[0]
AIC3 = -2*solution3 + (2*5)
BIC3 = -2*solution3 + 5*ln(21)
print("3-segment lambda1: {}".format(lambda13))
print("3-segment lambda2: {}".format(lambda23))
print("3-segment lambda3: {}".format(lambda33))
print("3-segment theta1:  {}".format(theta13))
print("3-segment theta2:  {}".format(theta23))
print("3-segment theta3:  0.0")
print("Max Log Likelihoood {}".format(solution3))
print("AIC {}".format(AIC3))
print("BIC {}".format(BIC3))

3-segment lambda1: 3.483324982796871
3-segment lambda2: 11.215822756574543
3-segment lambda3: 0.2905543120286637
3-segment theta1:  0.6744313253861919
3-segment theta2:  -0.4304098028987423
3-segment theta3:  0.0
Max Log Likelihoood -1132.0429842756384
AIC 2274.085968551277
BIC 2279.308580739894


In [8]:
def segments4(params,a,b):
    t1,t2,t3,lam1,lam2,lam3,lam4 = params
    pi1 = (e**t1)/((e**t1)+(e**t2)+(e**t3)+1) # calculating pie values(pie1,pie2,pie3)
    pi2 = (e**t2)/((e**t1)+(e**t2)+(e**t3)+1)
    pi3 = (e**t3)/((e**t1)+(e**t2)+(e**t3)+1)
    pi4 = (1)/((e**t1)+(e**t2)+(e**t3)+1)
    ll = 0
    for i in range(len(b)):
        ll += a[i]*ln(pi1*(poisson.pmf(b[i],lam1)) + pi2*(poisson.pmf(b[i],lam2)) + pi3*(poisson.pmf(b[i],lam3))+pi4*(poisson.pmf(b[i],lam4))) # as per the formula given in the slides 
    return (-1)*ll  
#same as above, but with 4-segments for the theta values

segments4sol = minimize(
    segments4,
    args = (a,b),
    x0 = np.array((1,2,3,1,1,1,1)),
    bounds=[(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

theta14 = segments4sol.x[0]
theta24 = segments4sol.x[1]
theta34 = segments4sol.x[2]
lambda14 = segments4sol.x[3]
lambda24 = segments4sol.x[4]
lambda34 = segments4sol.x[5]
lambda44 = segments4sol.x[6]
solution = -1*segments4sol.fun[0]
seg4AIC = -2*(solution) + (2*7)
seg4BIC = -2*(solution) + 7*ln(21)
print("lambda1: {}".format(lambda14))
print("lambda2: {}".format(lambda24))
print("lambda3: {}".format(lambda34))
print("lambda4: {}".format(lambda44))
print("theta1:  {}".format(theta14))
print("theta2:  {}".format(theta24))
print("theta3:  {}".format(theta34))
print("theta4:  0.0")
print("Maximum Log likelihood {}".format(solution))
print("AIC {}".format(seg4AIC))
print("BIC {}".format(seg4BIC))

lambda1: 7.416397934454427
lambda2: 0.20468598668769653
lambda3: 12.871224266891193
lambda4: 3.0016430670464347
theta1:  -1.2000216230440717
theta2:  -0.722007957028929
theta3:  -1.5974750982718735
theta4:  0.0
Maximum Log likelihood -1130.0705917767114
AIC 2274.1411835534227
BIC 2281.452840617487


In [9]:
#we want to make three functions, each enabling us to find the likiehood of a certian amount of purchases per customer per week
def seg2predict(params, b):
    pi,lam1,lam2 = params
    totalprob = sum([(pi*poisson.pmf(b,lam1)), ((1-pi)*poisson.pmf(b,lam2))])
    prob1 = (pi*poisson.pmf(b,lam1))/totalprob
    prob2 = ((1-pi)*poisson.pmf(b,lam2))/totalprob
    purch1 = lam1*prob1
    purch2 = lam2*prob2
    totalpurch = purch1 + purch2
    return totalpurch


def seg3predict(params, b):
    t1,t2,lam1,lam2,lam3 = params
    pi1 = (e**t1)/((e**t1)+(e**t2)+1) # calculating pie values(pie1,pie2,pie3)
    pi2 = (e**t2)/((e**t1)+(e**t2)+1)
    pi3 = (1)/((e**t1)+(e**t2)+1)
    totalprob = sum([(pi1*poisson.pmf(b,lam1)), (pi2*poisson.pmf(b,lam2)), (pi3*poisson.pmf(b,lam3))])
    prob1 = (pi1*poisson.pmf(b,lam1))/totalprob
    prob2 = (pi2*poisson.pmf(b,lam2))/totalprob
    prob3 = (pi3*poisson.pmf(b,lam3))/totalprob
    purch1 = lam1*prob1
    purch2 = lam2*prob2
    purch3 = lam3*prob3
    totalpurch = purch1 + purch2 + purch3
    return totalpurch

def seg4predict(params,b):
    t1,t2,t3,lam1,lam2,lam3,lam4 = params
    pi1 = (e**t1)/((e**t1)+(e**t2)+(e**t3)+1) 
    pi2 = (e**t2)/((e**t1)+(e**t2)+(e**t3)+1)
    pi3 = (e**t3)/((e**t1)+(e**t2)+(e**t3)+1)
    pi4 = (1)/((e**t1)+(e**t2)+(e**t3)+1)
    totalprob = sum([(pi1*poisson.pmf(b,lam1)), (pi2*poisson.pmf(b,lam2)), (pi3*poisson.pmf(b,lam3)), (pi4*poisson.pmf(b,lam4))])
    prob1 = (pi1*poisson.pmf(b,lam1))/totalprob
    prob2 = (pi2*poisson.pmf(b,lam2))/totalprob
    prob3 = (pi3*poisson.pmf(b,lam3))/totalprob
    prob4 = (pi4*poisson.pmf(b,lam4))/totalprob
    purch1 = lam1*prob1
    purch2 = lam2*prob2
    purch3 = lam3*prob3
    purch4 = lam4*prob4
    totalpurch = purch1 + purch2 + purch3 + purch4
    return totalpurch

#Predicting 5 last week
#2-segment
params2 = seg2pi, seg2lam1, seg2lam2
seg2amt = 8*seg2predict(params2, 5)

#3-segment
params3 = theta13, theta23, lambda13, lambda23, lambda33
seg3amt = 8*seg3predict(params3, 5)

#4-segment
params4 = theta14, theta24, theta34, lambda14, lambda24, lambda34, lambda44
seg4amt = 8*seg4predict(params4, 5)

print("2-segment: ", seg2amt)
print("3-segment: ", seg3amt)
print("4-segment: ", seg4amt)

2-segment:  42.78113482750646
3-segment:  30.82525089245209
4-segment:  33.67308637118602


In [10]:
#Predicting 8 last week
#2-segment
params2 = seg2pi, seg2lam1, seg2lam2
seg2amt8 = 8*seg2predict(params2, 9)

#3-segment
params3 = theta13, theta23, lambda13, lambda23, lambda33
seg3amt8 = 8*seg3predict(params3, 9)

#4-segment
params4 = theta14, theta24, theta34, lambda14, lambda24, lambda34, lambda44
seg4amt8 = 8*seg4predict(params4, 9)

print("2-segment: ", seg2amt8)
print("3-segment: ", seg3amt8)
print("4-segment: ", seg4amt8)

2-segment:  72.87063023453766
3-segment:  80.06345719960211
4-segment:  69.4530301727579


# Part 2

In [11]:
df = pd.read_csv('C:/Users/anushatc/Downloads/articles(1).csv')
df

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26
...,...,...,...,...,...,...
910,11,0,1,2,2.86,7
911,12,0,1,1,4.29,35
912,12,0,1,1,1.86,5
913,16,0,1,0,1.74,21


In [12]:
#Part 2 Question 1 - Poisson Regression

a = np.array(df[['female','married'	,'kids','prestige','menpubs']])
b = np.array(df[['articles']])

def funa(params,a,b):
    sum_funa = 0
    for i in range(len(b)):
        ans = 0
        for j in range(len(a[i])):
            ans += params[j]*a[i][j]
        ans = np.exp(ans)
        ans = np.log(params[j+1]*ans) 
        sum_funa += b[i]*ans
    return sum_funa

def funb(params,a):
    sum_funb = 0
    for i in range(len(a)):
        ans = 0
        for j in range(len(a[i])):
            ans += params[j]*a[i][j]
        sum_funb += e**(ans)
    return params[len(params)-1]*sum_funb


def func(b):
    ln = np.log
    sum_func = 0
    for i in b:
        sum_func += ln(factorial(i))
    return sum_func

def PLL(params,a,b):
    return -1*(funa(params,a,b)-funb(params,a)-func(b))
#these series of four functions break the Poission regression into it's three component parts
#Each part will iterate over all the variables in the dataset with given paramaters, and return a single summationed log likelihood
#Then, these will combine to get total log likelihood

soln = minimize(
    PLL,
    args = (a,b),
    x0 = np.array((0.1,0.1,0.1,0.1,0.1,1)),
    bounds=[(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8}
)
beta1 = soln.x[0]
beta2 = soln.x[1]
beta3 = soln.x[2]
beta4 = soln.x[3]
beta5 = soln.x[4]
lambda0 = soln.x[5]
poissionLL = -1*soln.fun[0]
poissonAIC = -2*(poissionLL) + (2*6)
poissonBIC = -2*(poissionLL) + 6*ln(915)

print("poission lambda0 {}".format(lambda0))
print("poission beta1(female) {}".format(beta1))
print("poission beta2(married) {}".format(beta2))
print("poission beta3(kids) {}".format(beta3))
print("poission beta4(prestige) {}".format(beta4))
print("poissonbeta5(menpubs) {}".format(beta5))
print("Maximum Log likelihood  {}".format(poissionLL))
print("AIC {}".format(poissonAIC))
print("BIC {}".format(poissonBIC))


poission lambda0 1.3560787638729377
poission beta1(female) -0.2246053289670147
poission beta2(married) 0.155227075526513
poission beta3(kids) -0.18488118293434616
poission beta4(prestige) 0.012830059828442866
poissonbeta5(menpubs) 0.025542874952492724
Maximum Log likelihood  -1651.0563163426154
AIC 3314.112632685231
BIC 3343.026177076884


In [13]:
#Part 2 Question 2 - NBD Regression
def funa(params,a,k):
    n = params[len(a)+1]
    parta = ln(gamma(n+k))
    partb = ln(gamma(n))
    partc = ln(factorial(k))
    return parta - (partb + partc)

def funb(params,a,k):
    alpha = params[len(a)]
    n = params[len(a)+1]
    beta_x_sum = 0
    for j in range(len(a)):
        beta_x_sum += params[j]*a[j]
    denom = e**beta_x_sum
    parta = n*(ln(alpha) - ln(alpha+denom))
    partb = k*(ln(denom) - ln(alpha+denom))
    return parta+partb

def NBDLL(params,a,b):
    LL = 0
    for i in range(len(b)):
        LL += funa(params,a[i],b[i]) + funb(params,a[i],int(b[i]))
    return (-1)*LL
#this breaks the NBD regression ito two parts. The first part calculates the gammas/beta distribution
#the second part calculates the other two "chunks" of the formula, where the regression exists


soln = minimize(
    NBDLL,
    args = (a,b),
    x0 = np.array((1,1,1,1,1,1,1)),
    bounds=[(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8}
)
beta1 = soln.x[0]
beta2 = soln.x[1]
beta3 = soln.x[2]
beta4 = soln.x[3]
beta5 = soln.x[4]
alpha = soln.x[5]
n = soln.x[6]
NBDSol = -1*soln.fun[0]
NBDAIC = -2*(NBDSol) + (2*7)
NBDBIC = -2*(NBDSol) + 7*ln(915)

print("NBD Alpha {}".format(alpha))
print("NBD Shape parameter(n) {}".format(n))
print("NBD beta1(female) {}".format(beta1))
print("NBD beta2(married) {}".format(beta2))
print("NBD beta3(kids) {}".format(beta3))
print("NBD beta4(prestige) {}".format(beta4))
print("NBD beta5(menpubs) {}".format(beta5))
print("Log likelihood NBD {}".format(NBDSol))
print("AIC NBD {}".format(NBDAIC))
print("BIC NBD {}".format(NBDBIC))

NBD Alpha 1.7525428859913976
NBD Shape parameter(n) 2.264262586074805
NBD beta1(female) -0.21649109322432747
NBD beta2(married) 0.15053811999676306
NBD beta3(kids) -0.17640525613034028
NBD beta4(prestige) 0.015247751277828188
NBD beta5(menpubs) 0.029083705063057366
Log likelihood NBD -1560.9583402036756
AIC NBD 3135.916680407351
BIC NBD 3169.6491488642796


In [14]:
def zifuna(params,a,k):
    n = params[len(a)+1]
    return gamma(n+k)/(gamma(n)*factorial(k))

def zifunb(params,a,k):
    n = params[len(a)+1]
    alpha = params[len(a)]
    betax_sum = 0
    for j in range(len(a)):
        betax_sum += params[j]*a[j]
    denom = e**betax_sum
    parta = (alpha/(alpha+denom))**n
    partb = (denom/(alpha+denom))**k
    return parta*partb

def funa(params,a,k):
    n = params[len(a)+1]
    parta = ln(gamma(n+k))
    partb = ln(gamma(n))
    partc = ln(factorial(k))
    return parta - (partb + partc)

def funb(params,a,k):
    alpha = params[len(a)]
    n = params[len(a)+1]
    beta_x_sum = 0
    for j in range(len(a)):
        beta_x_sum += params[j]*a[j]
    denom = e**beta_x_sum
    parta = n*(ln(alpha) - ln(alpha+denom))
    partb = k*(ln(denom) - ln(alpha+denom))
    return parta+partb

def NBDinfREG(params,a,b):
    pi = params[len(params)-1]
    ll = 0
    for i in range(len(b)):
        k = int(b[i])
        if b[i] == 0:
            ll += ln(pi + ((1-pi)*(zifuna(params,a[i],k))*(zifunb(params,a[i],k))))
        else:
            ll += ln((1-pi)) + funa(params,a[i],k) + funb(params,a[i],k)
    return -1*(ll)

soln = minimize(
    NBDinfREG,
    args = (a,b),
    x0 = np.array((0.1,0.1,0.1,0.1,0.1,1,1,0.01)),
    bounds=[(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,None),(0.000001,None),(0.000001,0.999999)],
    tol=1e-10,
    options={'ftol' : 1e-8}
)

beta1 = soln.x[0]
beta2 = soln.x[1]
beta3 = soln.x[2]
beta4 = soln.x[3]
beta5 = soln.x[4]
alpha = soln.x[5]
n = soln.x[6]
pie = soln.x[7]
ll0infreg = -1*soln.fun
regAIC = -2*(ll0infreg) + (2*8)
regBIC = -2*(ll0infreg) + 8*ln(915)

print("0-inflated NBD Reg Alpha {}".format(alpha))
print("0-inflated NBD Reg Shape parameter(n) {}".format(n))
print("0-inflated NBD Reg Pi {}".format(pie))
print("0-inflated NBD Reg beta1(female) {}".format(beta1))
print("0-inflated NBD Reg beta2(married) {}".format(beta2))
print("0-inflated NBD Reg beta3(kids) {}".format(beta3))
print("0-inflated NBD Reg beta4(prestige) {}".format(beta4))
print("0-inflated NBD Reg beta5(menpubs) {}".format(beta5))
print("0-inflated NBD Reg Log likelihood {}".format(ll0infreg))
print("0-inflated NBD Reg AIC {}".format(regAIC))
print("0-inflated NBD Reg BIC {}".format(regBIC))

0-inflated NBD Reg Alpha 1.7526942673415973
0-inflated NBD Reg Shape parameter(n) 2.264313846527012
0-inflated NBD Reg Pi 1e-06
0-inflated NBD Reg beta1(female) -0.2164607659475409
0-inflated NBD Reg beta2(married) 0.1507767826971019
0-inflated NBD Reg beta3(kids) -0.17670344839573615
0-inflated NBD Reg beta4(prestige) 0.015258272439650505
0-inflated NBD Reg beta5(menpubs) 0.02908440779500091
0-inflated NBD Reg Log likelihood -1560.9584036439323
0-inflated NBD Reg AIC 3137.9168072878647
0-inflated NBD Reg BIC 3176.4681998100687


In [15]:
print("Ding! The models have all ran. It was nice taking this class!")

Ding! The models have all ran. It was nice taking this class!
